<div style="display:block; margin-bottom:50px">
<h1 style="margin-bottom:25px; margin-top: 25px; font-size:3rem;color:#4c76ce;text-align:center;">
    Praktinis Projektas: Gapminder Duomenų Vizualizacija</h1>
    
<h2 style="margin-bottom: 25px;font-size:2.5rem;text-align:center;color:#8d1c1a;">
    3. Gapminder Vizualizacija
    </h2>
       
<img src="https://raw.githubusercontent.com/lajmcourses/Images/master/bokeh.png"
     style="position:absolute;top:5px;left:25px;height:150px;width:auto;margin-bottom:35px;">
</div>

In [9]:
import numpy as np
import pandas as pd

from bokeh.layouts import layout
from bokeh.models import ColumnDataSource, Label
from bokeh.models import Button, CustomJS, Slider
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import factor_cmap

output_notebook()

pd.options.display.precision = 2
pd.options.display.float_format = '{:.2f}'.format


Loading BokehJS ...

In [10]:
# Plot Settings

TITLE = "\nGapminder: The Third World Countries and the Western World\n"
YEAR_RANGE = range(1960, 2021)


start_year = min(YEAR_RANGE)
end_year = max(YEAR_RANGE)

In [11]:
# Data

In [13]:
# Plot
    
p = figure(height=650, width=900)


# Title
p.title = TITLE
p.title.align = "center"
p.title.text_font_size = "25px"
p.title.text_font_style = "normal"
p.title.text_color = "steelblue"


# Slider
year_slider = Slider(
                start=start_year, 
                end=end_year, 
                value=start_year, 
                step=1, 
                width=450,
                margin=(15, 15),
                title="Year")


sld_callback = CustomJS(args=dict(),
                    code="console.log(cb_obj.value)")
year_slider.js_on_change("value", sld_callback);


# Plot layout
plot_layout = layout(
    [
        [p],
        [year_slider]
    ]
)

show(plot_layout)